In [1]:
## Set-up Environment
import os
import sys

# Set project root (assumes notebook is in 'notebooks/' and project is one level up)
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
os.chdir(project_root)

# Add root to Python path for absolute imports like `env.trading_env`
if project_root not in sys.path:
    sys.path.append(project_root)

print("📁 Notebook directory:", notebook_dir)
print("✅ Project root set to:", project_root)


📁 Notebook directory: /workspaces/Strategy_PPO_Bots
✅ Project root set to: /workspaces


In [6]:
%cd /workspaces/Strategy_PPO_Bots


/workspaces/Strategy_PPO_Bots


In [7]:
## Check to ensure in the right directory
! pwd

/workspaces/Strategy_PPO_Bots


In [8]:
## Install dependencies
! pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
## Run integration test script
#! pytest tests/integration/test_integration.py --maxfail=1 --disable-warnings -q

In [9]:
## Generate Training Data
! python data_gen/generate_data.py --tickers-file data_gen/tickers.txt --interval 5min --start 2023-01-01 --end 2023-08-31

Fetching NVDA from 2023-01-01 to 2023-08-31 (5 minute)...
Saved NVDA_5minute to data/NVDA_5minute.csv
Fetching NVDA from 2023-01-01 to 2023-08-31 (1 day)...
Saved NVDA_1day to data/NVDA_1day.csv
Fetching NVDA from 2023-01-01 to 2023-08-31 (1 week)...
Saved NVDA_1week to data/NVDA_1week.csv


In [5]:
!ls -R /workspaces


/workspaces:
Strategy_PPO_Bots

/workspaces/Strategy_PPO_Bots:
 README.md	      env		   requirements.txt
 callbacks	     'et --hard d0b1daa'   run_with_config.sh
 config.json	      eval		   tests
 data		      logs		   train
 data_gen	      mlruns		   training_package_standalone.zip
 docker-compose.yml   models
 dockerfile	      notebooks

/workspaces/Strategy_PPO_Bots/callbacks:
__init__.py  __pycache__  performance_metrics_callback.py

/workspaces/Strategy_PPO_Bots/callbacks/__pycache__:
__init__.cpython-310.pyc  performance_metrics_callback.cpython-310.pyc
__init__.cpython-311.pyc  performance_metrics_callback.cpython-311.pyc

/workspaces/Strategy_PPO_Bots/data:
NVDA_1day.csv  NVDA_1week.csv  NVDA_5minute.csv  test_NVDA_5minute.csv

/workspaces/Strategy_PPO_Bots/data_gen:
generate_data.py  tickers.txt

/workspaces/Strategy_PPO_Bots/env:
__init__.py  action_handler.py	     observation_builder.py  trading_env.py
__pycache__  feature_engineering.py  reward_calculator.py

/workspaces/Str

In [ ]:
## ─── Plot all 1day CSVs in data/ ─────────────────────────────────────────
import glob

import matplotlib.pyplot as plt
import pandas as pd

# 1. Grab every CSV matching *_1day.csv
csv_files = glob.glob("data/*_1day.csv")
if not csv_files:
    raise FileNotFoundError("No files found: data/*_1day.csv")

# 2. Loop and plot
for file_path in sorted(csv_files):
    # Load and parse time
    df = pd.read_csv(file_path, parse_dates=[0])
    df.set_index(df.columns[0], inplace=True)

    # Extract ticker from filename
    ticker = file_path.split("/")[-1].split("_")[0]

    # Create the figure
    plt.figure(figsize=(10, 4))
    plt.plot(df.index, df["close"], linewidth=1)
    plt.title(f"{ticker} 1-Day Close Price")
    plt.xlabel("Time")
    plt.ylabel("Close Price")
    plt.tight_layout()
    plt.show()

In [ ]:
## Run Integration Test for PPO
# %run train/train_ppo.py --test


In [ ]:
import torch

print("CUDA Available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


In [ ]:
# Run PPO training using files in `data/` directory
%run train/train_ppo.py

In [ ]:
! ls ../debug_rewards.log
